In [43]:
import pandas as pd
import requests
import bs4

from threading import Thread
from queue import Queue
import json

In [3]:
data = pd.read_csv("data/DILI.csv")

In [3]:
data["DILI_CONCERN"].value_counts()

Less-DILI-Concern    183
Most-DILI-Concern    116
No-DILI-Concern       88
Less-DILI-concern      5
No-DILI-concern        4
Name: DILI_CONCERN, dtype: int64

In [ ]:
results = {}
errors = []

In [80]:
concurrent = 10

def do_work():
    while True:
        name = q.get()
        
        res = requests.get(f"https://zinc15.docking.org/substances/search/?q={name}")
        soup = bs4.BeautifulSoup(res.content)
        try:
            href = soup.find(lambda x: x.name=="a" and "title" in x.attrs and "More about" in x["title"]).attrs['href']
            extract_smiles(name, href)
        except AttributeError:
            try:
                smiles = extract_from_pubmed(name)
                results[name] = smiles
                print(name, smiles)
            except AttributeError:
                print("Err!", name)
        finally:
            q.task_done()

def extract_smiles(name, href):
    res = requests.get(f"https://zinc15.docking.org{href}")
    soup = bs4.BeautifulSoup(res.content)
    smiles = soup.find(id="substance-smiles-field").attrs['value']
    results[name] = smiles
    print(name, smiles)
    
def extract_from_pubmed(name):
    res = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/compound/{name}")
    soup = bs4.BeautifulSoup(res.content)
    cid = soup.find(lambda x: "name" in x.attrs and x.attrs["name"]=="pubchem_uid_value").attrs["content"]
    
    req_data = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{cid}/JSON/").content
    req_data = json.loads(req_data)
    for seq in req_data['Record']['Section']:
        if seq["TOCHeading"]=="Names and Identifiers":
            for subseq in seq["Section"]:
                if subseq["TOCHeading"]=="Computed Descriptors":
                    for subsubseq in subseq["Section"]:
                        if subsubseq["TOCHeading"]=="Canonical SMILES":
                            return subsubseq["Information"][0]["Value"]["StringWithMarkup"][0]["String"]

q = Queue(concurrent * 2)
for i in range(concurrent):
    t = Thread(target=do_work)
    t.daemon = True
    t.start()
try:
    for url in data["COMPOUND_NAME"]:
        if url in results:
            continue
        q.put(str(url).strip())
    q.join()
except KeyboardInterrupt:
    sys.exit(1)

Err! sacrosidase
Err! ferumoxytol
Err! protamine sulfate
Err! gemtuzumab


In [81]:
data["SMILES"] = data["COMPOUND_NAME"].apply(lambda x: results[x] if x in results else '')
data.to_csv("data/processed_DILI.csv")

In [82]:
(data["SMILES"].str.len() > 0).sum()

396